In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import random
import copy
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import RACETrack
rt = RACETrack()

In [ ]:
total_nodes, layout_nodes = 20, 20
nodes, to_adjust = [], []
for i in range(total_nodes): nodes.append('node_' + str(i))
_lu_, pos = {'fm':[], 'to':[], 'ct':[]}, {}
for i in range(len(nodes)):
    _lu_['fm'].append(nodes[i]), _lu_['to'].append(random.choice(nodes)), _lu_['ct'].append(1)
    _lu_['fm'].append(nodes[i]), _lu_['to'].append(random.choice(nodes)), _lu_['ct'].append(1)
    _lu_['fm'].append(nodes[i]), _lu_['to'].append(random.choice(nodes)), _lu_['ct'].append(1)
    _lu_['fm'].append(nodes[i]), _lu_['to'].append(random.choice(nodes)), _lu_['ct'].append(1)

    j = random.randint(0,3)
    if   j == 0: pos[nodes[i]] = (0, random.random()*100)
    elif j == 1: pos[nodes[i]] = (random.random()*100, 0)
    elif j == 2: pos[nodes[i]] = (100, random.random()*100)
    else:        pos[nodes[i]] = (random.random()*100, 100)
    if len(to_adjust) < layout_nodes: to_adjust.append(nodes[i])

a_attach, b_attach, c_attach, d_attach, e_attach_1, e_attach_2 = nodes[0], nodes[1], nodes[2], nodes[3], nodes[4], nodes[5]
pos['corner_a'] = (0,   50)
pos['corner_b'] = (50,  100)
pos['corner_c'] = (100, 50)
pos['corner_d'] = (50,  0)
pos['e']        = (0,   0)
_lu_['fm'].append(a_attach),   _lu_['to'].append('corner_a'), _lu_['ct'].append(10)
_lu_['fm'].append(b_attach),   _lu_['to'].append('corner_b'), _lu_['ct'].append(10)
_lu_['fm'].append(c_attach),   _lu_['to'].append('corner_c'), _lu_['ct'].append(10)
_lu_['fm'].append(d_attach),   _lu_['to'].append('corner_d'), _lu_['ct'].append(10)
_lu_['fm'].append(e_attach_1), _lu_['to'].append('e'),        _lu_['ct'].append(10)
_lu_['fm'].append(e_attach_2), _lu_['to'].append('e'),        _lu_['ct'].append(10)

df  = pl.DataFrame(_lu_)
g   = rt.createNetworkXGraph(df, [('fm','to')])
_link_orig_ = rt.link(df, [('fm','to')], pos=pos, w=512, h=512)
_adj_       = rt.circularOptimizedArrangement(g, to_adjust, pos, xy=(50.0,50.0), r=25.0)
new_pos     = copy.deepcopy(pos)
for _node_ in _adj_: new_pos[_node_] = _adj_[_node_]
_link_adj_  = rt.link(df, [('fm','to')], count_by='ct', link_size='vary', pos=new_pos, w=512, h=512)
rt.tile([_link_orig_, _link_adj_])

In [ ]:
ln_params = {'relationships':[('fm','to')], 'pos':new_pos, 'draw_labels':False, 'count_by':'ct', 'link_size':'vary'}
_rtg_ = rt.interactiveGraphLayout(df, ln_params, w=512, h=512)
_rtg_